In [ ]:
# Clone the repo (safe to re-run if it already exists)
import os

if not os.path.exists("Loan-Approval-Prediction---New-York---2025"):
    !git clone https://github.com/remussamoila/Loan-Approval-Prediction---New-York---2025.git

%cd Loan-Approval-Prediction---New-York---2025


# ⚙️ Advanced Modeling - Loan Approval



[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](
https://colab.research.google.com/github/remussamoila/Loan-Approval-Prediction---New-York---2025/blob/main/notebooks/04_advanced_models.ipynb)

# 📘 04_advanced_models.ipynb

**Goal**: Train and compare advanced models (XGBoost and LightGBM) for the SME Loan Approval task. Evaluate with Mean F1 Score.

This notebook includes:
- Data loading and preprocessing
- Training XGBoost and LightGBM
- Hyperparameter tuning (GridSearchCV)
- Model comparison and export

In [ ]:
# ✅ Step 1: Load libraries
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# ✅ Step 2: Load and preprocess data
df = pd.read_csv('data/train.csv', low_memory=False)
df['DisbursementGross'] = df['DisbursementGross'].replace('[\$,]', '', regex=True).astype(float)
df['BalanceGross'] = df['BalanceGross'].replace('[\$,]', '', regex=True).astype(float)
df.drop(columns=['id', 'LoanNr_ChkDgt', 'Name', 'City', 'DisbursementDate', 'ApprovalDate'], inplace=True)
df.fillna(-1, inplace=True)
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].astype(str)
    df[col] = LabelEncoder().fit_transform(df[col])
X = df.drop(columns=['Accept'])
y = df['Accept']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [ ]:
# ✅ Step 2B: Train Baseline Models
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

# Dummy Classifier
dummy = DummyClassifier(strategy="most_frequent")
dummy.fit(X_train, y_train)
f1_dummy = f1_score(y_val, dummy.predict(X_val))

# Logistic Regression
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)
f1_lr = f1_score(y_val, lr.predict(X_val))

# Decision Tree
tree = DecisionTreeClassifier(random_state=42)
tree.fit(X_train, y_train)
f1_tree = f1_score(y_val, tree.predict(X_val))

print("📊 Dummy F1:", f1_dummy)
print("📊 Logistic Regression F1:", f1_lr)
print("📊 Decision Tree F1:", f1_tree)

In [ ]:
# ✅ Step 3: Train XGBoost
model_xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
model_xgb.fit(X_train, y_train)
y_pred_xgb = model_xgb.predict(X_val)
f1_xgb = f1_score(y_val, y_pred_xgb)
print('XGBoost F1 Score:', f1_xgb)

In [ ]:
# ✅ Step 4: Train LightGBM
model_lgb = LGBMClassifier(random_state=42)
model_lgb.fit(X_train, y_train)
y_pred_lgb = model_lgb.predict(X_val)
f1_lgb = f1_score(y_val, y_pred_lgb)
print('LightGBM F1 Score:', f1_lgb)

In [ ]:
# ✅ Step 5A: Hyperparameter Tuning (XGBoost)
param_grid_xgb = {
    'n_estimators': [100, 200],
    'max_depth': [4, 6],
    'learning_rate': [0.05, 0.1]
}
grid_xgb = GridSearchCV(XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
                        param_grid_xgb, scoring='f1', cv=3)
grid_xgb.fit(X_train, y_train)
print('Best XGBoost params:', grid_xgb.best_params_)
print('Best XGBoost F1:', grid_xgb.best_score_)

In [ ]:
# ✅ Step 5B: Hyperparameter Tuning (LightGBM)
param_grid_lgb = {
    'n_estimators': [100, 200],
    'max_depth': [4, 6],
    'learning_rate': [0.05, 0.1]
}
grid_lgb = GridSearchCV(LGBMClassifier(), param_grid_lgb, scoring='f1', cv=3)
grid_lgb.fit(X_train, y_train)
print('Best LightGBM params:', grid_lgb.best_params_)
print('Best LightGBM F1:', grid_lgb.best_score_)

In [ ]:
# ✅ Step 6: Compare Models
results = pd.DataFrame({
    'Model': ['Dummy', 'Logistic Regression', 'Decision Tree', 'XGBoost', 'LightGBM'],
    'F1 Score': [f1_dummy, f1_lr, f1_tree, f1_xgb, f1_lgb]
})
sns.barplot(x='Model', y='F1 Score', data=results)
plt.title('F1 Score Comparison (Baseline vs Advanced)')
plt.ylim(0, 1)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
results

In [ ]:
# ✅ Step 7: Export the best model
import joblib

# Assuming LightGBM gave the best results — adjust if needed
best_model = model_lgb if f1_lgb > f1_xgb else model_xgb
joblib.dump(best_model, 'models/best_model.joblib')
print("✅ Best model saved to models/best_model.joblib")